<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timegan_exchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install synthcity

# Training

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader
from synthcity.utils.serialization import save_to_file, load_from_file

log.add(sink=sys.stderr, level="INFO")

Mounted at /content/drive
[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [3]:
file_path = "/content/drive/Shareddrives/sp_env/datasets/Exchange Rate/exchange_rate.txt"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print('missing values:', df.isnull().sum())

   0.785500  1.611000  0.861698  0.634196  0.211242  0.006838  0.593000  \
0    0.7818    1.6100  0.861104  0.633513  0.211242  0.006863    0.5940   
1    0.7867    1.6293  0.861030  0.648508  0.211242  0.006975    0.5973   
2    0.7860    1.6370  0.862069  0.650618  0.211242  0.006953    0.5970   
3    0.7849    1.6530  0.861995  0.656254  0.211242  0.006940    0.5985   
4    0.7866    1.6537  0.861030  0.654879  0.211242  0.006887    0.6040   

   0.525486  
0  0.523972  
1  0.526316  
2  0.523834  
3  0.527426  
4  0.526177  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7587 entries, 0 to 7586
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0.785500  7587 non-null   float64
 1   1.611000  7587 non-null   float64
 2   0.861698  7587 non-null   float64
 3   0.634196  7587 non-null   float64
 4   0.211242  7587 non-null   float64
 5   0.006838  7587 non-null   float64
 6   0.593000  7587 non-null   float64
 7   0.5

In [4]:
# for exchange dataset only
df.columns = [
    'Australia', 'British_Pound', 'Canada', 'Switzerland',
    'China', 'Japan', 'New_Zealand', 'Singapore'
]

# set seed and randomly choose 2 features
np.random.seed(42)

# Filter the dataframe to only include selected features
selected_features = np.random.choice(df.columns, size=2, replace=False)
print(f"Selected features: {selected_features}")

# Immediately filter your full DataFrame down to just those two series:
df = df[selected_features]

Selected features: ['British_Pound' 'Japan']


In [7]:
# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training (for TimeGAN), 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # use later for LSTM-TSTR

# Normalize the data
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)


# set sequence length to 30 to capture monthly patterns
# dataset = daily
sequence_length = 30
temporal_data = []
observation_times = []

for start in range(len(df_scaled_train) - sequence_length + 1):
    sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data.append(sequence)
    observation_times.append(list(range(sequence_length)))  # relative time within the window


# Dummy outcome for data loader
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

# --- Create DataLoader for TimeGAN ---
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,  # List of sequences (DataFrames)
    observation_times=observation_times,  # List of time indices (DataFrames)
    static_data=None,  # No static data for now (can be set if needed)
    outcome=dummy_outcome,  # Dummy outcome for forecasting
)

# Print the loader info
print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")

TimeSeriesDataLoader created with 3471 sequences


In [8]:
print(len(df_train))  # Check the length of the dataframe
print(loader.dataframe())

3500
        seq_id  seq_time_id  seq_temporal_British_Pound  seq_temporal_Japan  \
0            0            0                    0.175634            0.327109   
1            0            1                    0.177668            0.321058   
2            0            2                    0.168709            0.315871   
3            0            3                    0.163281            0.314661   
4            0            4                    0.165788            0.318638   
...        ...          ...                         ...                 ...   
104125    3470           25                    0.325966            0.882434   
104126    3470           26                    0.327503            0.875692   
104127    3470           27                    0.313072            0.865318   
104128    3470           28                    0.312271            0.866010   
104129    3470           29                    0.305651            0.874308   

        seq_out_outcome  
0                   

In [9]:
from synthcity.plugins import Plugins

hparams = {
          "mode": "LSTM",
}

# Load TimeGAN with custom parameters
syn_model = Plugins().get("timegan", **hparams)

[2025-04-26T13:20:23.621941+0000][2010][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-04-26T13:20:23.621941+0000][2010][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py


In [10]:
# Print all parameters of initialized model
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeGANPlugin
clipping_value: 0
compress_dataset: False
dataloader_sampling_strategy: imbalanced_time_censoring
device: cuda
discriminator_batch_norm: False
discriminator_dropout: 0.1
discriminator_loss: None
discriminator_lr: 0.001
discriminator_n_iter: 1
discriminator_n_layers_hidden: 3
discriminator_n_units_hidden: 300
discriminator_nonlin: leaky_relu
discriminator_weight_decay: 0.001
embedding_penalty: 10
encoder: None
encoder_max_clusters: 20
expecting_conditional: False
fitted: False
gamma_penalty: 1
generator_batch_norm: False
generator_dropout: 0.01
generator_loss: None
generator_lr: 0.001
generator_n_layers_hidden: 2
generator_n_units_hidden: 150
generator_nonlin: leaky_relu
generator_nonlin_out_continuous: tanh
generator_nonlin_out_discrete: softmax
generator_residual: True
generator_weight_decay: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timegan
module_relative_path: ../time_series/plugin_timegan.py
moments_penalty: 100
n_i

## fitting the model

In [12]:
print(loader.shape)

# --- Train the model ---
syn_model.fit(loader)

(104130, 5)


100%|██████████| 1000/1000 [1:56:52<00:00,  7.01s/it]


In [20]:
#@title optional: generate data from saved model
from synthcity.utils.serialization import save_to_file, load_from_file

# Save model to drive
save_to_file('/content/drive/Shareddrives/sp_env/saved_models/GAN_Exchange.pkl', syn_model)

# Load the model
# syn_model = load_from_file('/content/drive/Shareddrives/sp_env/synmodel.pkl')

In [14]:
# --- Generate Synthetic Data ---
n_samples = len(temporal_data)
syn_data = syn_model.generate(count=n_samples)
print(syn_data.shape)

(100017, 5)


In [15]:
# --- Save with automated format ---
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-3500.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

Synthetic data saved to: /content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange/042625-152949-timeganplugin-3500.csv


# Evaluation

## Prerequisites

In [16]:
# --- Convert to NumPy
real_data = loader.dataframe().to_numpy()
synthetic_data = syn_data.dataframe().to_numpy()

# Define selected columns explicitly
# selected_columns = ['seq_temporal_HUFL', 'seq_temporal_HULL', 'seq_temporal_LUFL', 'seq_temporal_LULL', 'seq_temporal_MUFL', 'seq_temporal_MULL', 'seq_temporal_OT']
selected_columns = ['seq_temporal_British_Pound','seq_temporal_Japan']
# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data.dataframe()[selected_columns].to_numpy()

In [17]:
print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

[[0.17563361 0.32710927]
 [0.1776684  0.32105809]
 [0.16870937 0.31587137]
 ...
 [0.3130719  0.86531812]
 [0.31227102 0.86600968]
 [0.30565084 0.87430844]] 
 ------------------------------------------------------- 
 [[0.25763111 0.18547966]
 [0.0866552  0.18546773]
 [0.08670829 0.18559386]
 ...
 [0.31743031 0.1863488 ]
 [0.25738601 0.18644113]
 [0.61835132 0.18653295]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(104130, 2) (100017, 2)


## Generate distance metrics

### Helper Functions

In [18]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [24]:
# Assuming df_scaled is the DataFrame containing your scaled ETD data

# Compute Wasserstein Distance
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
print("Wasserstein Distance Results:")
print(wasserstein_results)

# Compute KL Divergence
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)
print("KL Divergence Results:")
print(kl_results)

(100017, 2) (100017, 2)
0.16271411492303786
0.21730924254524583
Wasserstein Distance Results:
{'seq_temporal_British_Pound': 0.16271411492303786, 'seq_temporal_Japan': 0.21730924254524583}
14.091844061719092
12.839344340461874
KL Divergence Results:
{'seq_temporal_British_Pound': 14.091844061719092, 'seq_temporal_Japan': 12.839344340461874}


# LSTM downstream

In [ ]:
from google.colab import drive
drive.mount('/content/drive')